先做GPT的变量

1. 先把几个文件给合并到一块

In [ ]:
import pandas as pd

# Define file paths
file_paths = {
    "df_241125": "/Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/result/241125advanced_multitopic_story_analysis.csv",
    "df_250202": "/Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/result/250202advanced_multitopic_story_analysis.csv",
    "df_241203": "/Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/result/241203advanced_multitopic_story_analysis_add.csv"
}

# Read CSV files into DataFrames
df_241125 = pd.read_csv(file_paths["df_241125"])
df_250202 = pd.read_csv(file_paths["df_250202"])
df_241203 = pd.read_csv(file_paths["df_241203"])


# Function to process the "故事名称" column
def process_story_name(df, column_name="故事名称"):
    df[column_name] = df[column_name].apply(lambda x: x.split("/")[-1].replace(".txt", ""))
    return df

# Process "故事名称" column in df_241203 and df_250202
df_241203 = process_story_name(df_241203)
df_250202 = process_story_name(df_250202)

# Merge df_241203 into df_241125 based on "故事名称"
df_merged = df_241125.merge(df_241203, on="故事名称", how="left", suffixes=('', '_241203'))

# Concatenate df_250202 vertically
df_final = pd.concat([df_merged, df_250202], axis=0, ignore_index=True)

# Handle duplicate column names
df_final = df_final.loc[:, ~df_final.columns.duplicated(keep='first')]

# Drop the last column (regardless of name)
df_final = df_final.iloc[:, :-1]

# Move "故事原文" column to the last position
if "故事原文" in df_final.columns:
    cols = [col for col in df_final.columns if col != "故事原文"] + ["故事原文"]
    df_final = df_final[cols]

df_final

,故事名称,仁义,忠诚,佛教,道教,儒家,杀戮,孝道,慈善,信任,...,宗族,祖先,平等,鬼神,古时候,馈赠,帝王,遵守,结拜,故事原文
0,01233__479-拿龙,A,A,D,D,D,D,D,A,A,...,D,D,D,A,D,D,D,A,D,479-拿 龙\n\n（回族）\n\n桃园坝子的回回人可多啦，相传回回人中还出了不少降龙伏虎...
1,01085__328-吐良的传说,A,A,D,D,D,A,D,D,D,...,D,D,D,D,A,D,D,D,D,328-吐良的传说\n\n（景颇族）\n\n传说在很久以前，瑞达崩地方有一个英俊剽悍的小伙子...
2,01362__611摇竹惊雀,C,D,D,D,D,D,D,D,C,...,D,D,D,D,A,D,D,B,C,611.摇竹惊雀\n\n（汉族）\n\n很早以前.在个旧锡矿山上，有一个叫金竹林的地方。这里...
3,01356__605-父亲长出了双牛脚,A,A,D,D,A,D,A,D,A,...,A,D,D,A,D,A,D,A,D,605-父亲长出了双牛脚\n\n（汉族）\n\n有一对相处得极好的朋友，一个叫张三，另一个叫...
4,01357__606偷盗的下场,A,A,D,D,A,D,A,A,C,...,A,D,D,A,A,A,D,A,D,606.偷盗的下场\n\n（阿昌族〉\n\n很久以前，石嘴山下住着两兄弟.哥哥叫岩?弟弟叫航...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20351,19404__235敬火神的传说,A,A,D,A,D,D,A,A,A,...,D,D,D,A,A,D,D,A,D,"235.敬火神的传说\n （鄂温克族）\n 在很早以前,有条河叫格尔必齐河，..."
20352,19457__288跳兔和甲鱼,D,D,D,D,D,D,D,D,C,...,D,D,D,D,D,C,D,D,D,288•跳兔和甲鱼\n （蒙古族）\n 一只甲鱼爬上河岸，遇见跳兔正在搬运食...
20353,19173__002银河的来历,D,D,C,D,D,A,D,D,D,...,D,A,D,D,A,D,D,D,D,002.银河的来历\n （蒙古族）\n 相传在遥远的古代，人类进行过一次空前...
20354,19778__622贪心的诺彦,D,D,D,D,D,D,D,D,D,...,D,D,D,D,A,D,D,D,D,622•贪心的诺彦\n （蒙古族）\n 从前有一个诺彦，他非常贪婪，想把整个...


In [16]:
df_final.to_excel("/Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/result/250203advanced_multitopic_story_analysis.xlsx")


2. 下面开始做GPT的变量

In [53]:
import pandas as pd

# 文件路径
story_analysis_path = "/Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/result/250203advanced_multitopic_story_analysis.xlsx"
spatial_join_path = "/Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/result/250202地图匹配&少数民族校对/250202地图匹配_spatial_join_2010_no_inconsistency.xlsx"
cgss_data_path = "/Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/result/241220to_use/CGSS2010-2015_withoutvarlabel.dta"

# 读取文件到DataFrame
gpt_story_analysis_df = pd.read_excel(story_analysis_path)
spatial_join_df = pd.read_excel(spatial_join_path)
cgss_data_df= pd.read_stata(cgss_data_path)

In [54]:
spatial_join_df['CNTYGB_2010'] = spatial_join_df['CNTYGB_2010'].fillna(spatial_join_df['matched_gbcode_2010'])

In [55]:
spatial_join_df.to_excel("/Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/result/250203地图匹配_spatial_join_2010_no_inconsistency.xlsx")

In [56]:
# 假设 gpt_story_analysis_df 和 spatial_join_df 已加载

# 合并数据框
spatial_join_df = pd.merge(
    spatial_join_df,
    gpt_story_analysis_df,
    left_on='name',
    right_on='故事名称',
    how='left'  # 左连接保留 spatial_join_df 的所有行
)

spatial_join_df.head()

,Unnamed: 0_x,rowid,name,cailu,provgb,citygb,cntygb,area,geocode,PROVGB_1953,...,宗族,祖先,平等,鬼神,古时候,馈赠,帝王,遵守,结拜,故事原文
0,0,1,04946__322老汉的宝物,missing values,missing valuesd,missing valuesd,missing valuesd,青海循化县,"{'lng': 116.413384, 'lat': 39.910925}",63.0,...,D,D,D,A,A,C,D,D,D,"322.老汉的宝物\n\n（撒拉族循化貝）\n\n从前,有个单身老汉，住在山里靠打猎过日子。..."
1,1,2,04743__121造过麦山的白土城,missing values,63,6321,632122,青海民和县,"{'lng': 102.83639, 'lat': 36.325561}",63.0,...,D,D,D,D,A,D,C,D,D,121.造过麦山的白土城\n\n（土族民和县）\n\n单阳公主用烈火打败了吐蕃兵以后，不幸又...
2,2,3,05119__496伊斯玛悔过,林昌林,63,6301,630100,青海西宁市,"{'lng': 101.78445, 'lat': 36.623385}",63.0,...,D,D,D,D,D,A,D,A,D,496.伊斯玛悔过\n\n（回族西宁市）\n\n在通往城镇的一条大路旁，住着一家老两口.老汉...
3,3,4,05049__425三姊妹,李友楼,missing valuesd,missing valuesd,missing valuesd,青海互助县,"{'lng': 101.964569, 'lat': 36.850022}",63.0,...,A,D,D,D,D,A,D,D,A,"425.三姊妹\n\n（土族互助县）\n\n有一对夫妻，妻子已经怀孕了,他俩盼望着能生一个男..."
4,4,5,04849__225老鼠的智慧,missing values,63,6323,632321,青海同仁县,"{'lng': 116.413384, 'lat': 39.910925}",63.0,...,D,D,D,D,D,A,D,A,D,225.老鼠的智慧\n\n（藏抜同仁县）\n\n有一次.老鼠、青蛙、狼，为争夺一件宝物争吵不...


In [41]:
spatial_join_df.columns

Index(['Unnamed: 0_x', 'rowid', 'name', 'cailu', 'provgb', 'citygb', 'cntygb',
       'area', 'geocode', 'PROVGB_1953', 'CITYGB_1953', 'CNTYGB_1953',
       'PROVGB_1964', 'CITYGB_1964', 'CNTYGB_1964', 'PROVGB_1982',
       'CITYGB_1982', 'CNTYGB_1982', 'PROVGB_1990a', 'CITYGB_1990a',
       'CNTYGB_1990a', 'PROVGB_2000', 'CITYGB_2000', 'CNTYGB_2000',
       'PROVGB_2010', 'CITYGB_2010', 'CNTYGB_2010', 'new_name', 'area_group',
       'centroid_1982', 'centroid_1990', 'centroid_2000', 'combined_centroid',
       'matched_gbcode_2010', 'matched_gbcode_1982', 'matched_gbcode_1990',
       'matched_gbcode_2000', 'Unnamed: 0_y', '故事名称', '仁义', '忠诚', '佛教', '道教',
       '儒家', '杀戮', '孝道', '慈善', '信任', '努力', '惩罚', '诚实', '长寿', '宗族', '祖先', '平等',
       '鬼神', '古时候', '馈赠', '帝王', '遵守', '结拜', '故事原文'],
      dtype='object')

In [42]:
spatial_join_df.head()

,Unnamed: 0_x,rowid,name,cailu,provgb,citygb,cntygb,area,geocode,PROVGB_1953,...,宗族,祖先,平等,鬼神,古时候,馈赠,帝王,遵守,结拜,故事原文
0,0,1,04946__322老汉的宝物,missing values,missing valuesd,missing valuesd,missing valuesd,青海循化县,"{'lng': 116.413384, 'lat': 39.910925}",63.0,...,D,D,D,A,A,C,D,D,D,"322.老汉的宝物\n\n（撒拉族循化貝）\n\n从前,有个单身老汉，住在山里靠打猎过日子。..."
1,1,2,04743__121造过麦山的白土城,missing values,63,6321,632122,青海民和县,"{'lng': 102.83639, 'lat': 36.325561}",63.0,...,D,D,D,D,A,D,C,D,D,121.造过麦山的白土城\n\n（土族民和县）\n\n单阳公主用烈火打败了吐蕃兵以后，不幸又...
2,2,3,05119__496伊斯玛悔过,林昌林,63,6301,630100,青海西宁市,"{'lng': 101.78445, 'lat': 36.623385}",63.0,...,D,D,D,D,D,A,D,A,D,496.伊斯玛悔过\n\n（回族西宁市）\n\n在通往城镇的一条大路旁，住着一家老两口.老汉...
3,3,4,05049__425三姊妹,李友楼,missing valuesd,missing valuesd,missing valuesd,青海互助县,"{'lng': 101.964569, 'lat': 36.850022}",63.0,...,A,D,D,D,D,A,D,D,A,"425.三姊妹\n\n（土族互助县）\n\n有一对夫妻，妻子已经怀孕了,他俩盼望着能生一个男..."
4,4,5,04849__225老鼠的智慧,missing values,63,6323,632321,青海同仁县,"{'lng': 116.413384, 'lat': 39.910925}",63.0,...,D,D,D,D,D,A,D,A,D,225.老鼠的智慧\n\n（藏抜同仁县）\n\n有一次.老鼠、青蛙、狼，为争夺一件宝物争吵不...


In [29]:
theme_columns = spatial_join_df.columns[-23:-1]  # 从第4列开始是主题列
theme_columns

Index(['仁义', '忠诚', '佛教', '道教', '儒家', '杀戮', '孝道', '慈善', '信任', '努力', '惩罚', '诚实',
       '长寿', '宗族', '祖先', '平等', '鬼神', '古时候', '馈赠', '帝王', '遵守', '结拜'],
      dtype='object')

In [57]:
import pandas as pd

# Define the columns of interest
columns_of_interest = ['仁义', '忠诚', '佛教', '道教', '儒家', '杀戮', '孝道', '慈善', '信任', '努力', 
                       '惩罚', '诚实', '长寿', '宗族', '祖先', '平等', '鬼神', '古时候', '馈赠', '帝王', 
                       '遵守', '结拜']

# Function to calculate proportion of each value (A, B, C, D)
def calculate_proportions(group):
    total_count = len(group)
    proportions = {}
    
    for col in columns_of_interest:
        if col in group.columns:
            for category in ['A', 'B', 'C', 'D']:
                proportions[f"gpt_proportion_{col}_{category}"] = (group[col] == category).sum() / total_count
    
    return pd.Series(proportions)

# Group by 'CNTYGB_2010' and apply the proportion function
result1 = spatial_join_df.groupby('CNTYGB_2010').apply(calculate_proportions).reset_index()
result1

/var/folders/w5/_ylk19595zl7_649y60st5680000gn/T/ipykernel_24497/162201407.py:21: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  result1 = spatial_join_df.groupby('CNTYGB_2010').apply(calculate_proportions).reset_index()


,CNTYGB_2010,gpt_proportion_仁义_A,gpt_proportion_仁义_B,gpt_proportion_仁义_C,gpt_proportion_仁义_D,gpt_proportion_忠诚_A,gpt_proportion_忠诚_B,gpt_proportion_忠诚_C,gpt_proportion_忠诚_D,gpt_proportion_佛教_A,...,gpt_proportion_帝王_C,gpt_proportion_帝王_D,gpt_proportion_遵守_A,gpt_proportion_遵守_B,gpt_proportion_遵守_C,gpt_proportion_遵守_D,gpt_proportion_结拜_A,gpt_proportion_结拜_B,gpt_proportion_结拜_C,gpt_proportion_结拜_D
0,110101.0,0.142857,0.047619,0.142857,0.666667,0.142857,0.047619,0.047619,0.761905,0.095238,...,0.047619,0.428571,0.000000,0.047619,0.285714,0.666667,0.000000,0.0,0.00,1.000000
1,110102.0,0.093023,0.023256,0.232558,0.651163,0.209302,0.000000,0.069767,0.720930,0.046512,...,0.046512,0.558140,0.046512,0.046512,0.302326,0.604651,0.046512,0.0,0.00,0.953488
2,110103.0,0.083333,0.000000,0.062500,0.854167,0.062500,0.000000,0.000000,0.937500,0.062500,...,0.062500,0.729167,0.104167,0.000000,0.291667,0.604167,0.020833,0.0,0.00,0.979167
3,110104.0,0.100000,0.000000,0.000000,0.900000,0.250000,0.000000,0.000000,0.750000,0.000000,...,0.100000,0.250000,0.150000,0.050000,0.150000,0.650000,0.000000,0.0,0.05,0.950000
4,110105.0,0.500000,0.000000,0.100000,0.400000,0.600000,0.000000,0.200000,0.200000,0.100000,...,0.000000,0.500000,0.100000,0.100000,0.500000,0.300000,0.000000,0.0,0.10,0.900000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1893,654322.0,0.800000,0.000000,0.200000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.800000,0.200000,0.000000,0.400000,0.400000,0.200000,0.0,0.00,0.800000
1894,654323.0,0.333333,0.000000,0.000000,0.666667,0.666667,0.000000,0.000000,0.333333,0.000000,...,0.000000,0.333333,0.000000,0.000000,0.333333,0.666667,0.000000,0.0,0.00,1.000000
1895,654324.0,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.428571,0.571429,0.000000,0.285714,0.142857,0.000000,0.0,0.00,1.000000
1896,654326.0,0.500000,0.000000,0.500000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.500000,0.500000,0.000000,0.000000,0.500000,0.000000,0.0,0.00,1.000000


In [58]:
wide_df = result1

In [59]:
wide_df['CNTYGB_2010'] = wide_df['CNTYGB_2010'].fillna(0).astype(int)

In [60]:
cgss_data_df['china10_cntygb'] = cgss_data_df['china10_cntygb'].fillna(0).astype(int)

In [61]:
wide_df

,CNTYGB_2010,gpt_proportion_仁义_A,gpt_proportion_仁义_B,gpt_proportion_仁义_C,gpt_proportion_仁义_D,gpt_proportion_忠诚_A,gpt_proportion_忠诚_B,gpt_proportion_忠诚_C,gpt_proportion_忠诚_D,gpt_proportion_佛教_A,...,gpt_proportion_帝王_C,gpt_proportion_帝王_D,gpt_proportion_遵守_A,gpt_proportion_遵守_B,gpt_proportion_遵守_C,gpt_proportion_遵守_D,gpt_proportion_结拜_A,gpt_proportion_结拜_B,gpt_proportion_结拜_C,gpt_proportion_结拜_D
0,110101,0.142857,0.047619,0.142857,0.666667,0.142857,0.047619,0.047619,0.761905,0.095238,...,0.047619,0.428571,0.000000,0.047619,0.285714,0.666667,0.000000,0.0,0.00,1.000000
1,110102,0.093023,0.023256,0.232558,0.651163,0.209302,0.000000,0.069767,0.720930,0.046512,...,0.046512,0.558140,0.046512,0.046512,0.302326,0.604651,0.046512,0.0,0.00,0.953488
2,110103,0.083333,0.000000,0.062500,0.854167,0.062500,0.000000,0.000000,0.937500,0.062500,...,0.062500,0.729167,0.104167,0.000000,0.291667,0.604167,0.020833,0.0,0.00,0.979167
3,110104,0.100000,0.000000,0.000000,0.900000,0.250000,0.000000,0.000000,0.750000,0.000000,...,0.100000,0.250000,0.150000,0.050000,0.150000,0.650000,0.000000,0.0,0.05,0.950000
4,110105,0.500000,0.000000,0.100000,0.400000,0.600000,0.000000,0.200000,0.200000,0.100000,...,0.000000,0.500000,0.100000,0.100000,0.500000,0.300000,0.000000,0.0,0.10,0.900000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1893,654322,0.800000,0.000000,0.200000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.800000,0.200000,0.000000,0.400000,0.400000,0.200000,0.0,0.00,0.800000
1894,654323,0.333333,0.000000,0.000000,0.666667,0.666667,0.000000,0.000000,0.333333,0.000000,...,0.000000,0.333333,0.000000,0.000000,0.333333,0.666667,0.000000,0.0,0.00,1.000000
1895,654324,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.428571,0.571429,0.000000,0.285714,0.142857,0.000000,0.0,0.00,1.000000
1896,654326,0.500000,0.000000,0.500000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.500000,0.500000,0.000000,0.000000,0.500000,0.000000,0.0,0.00,1.000000


In [62]:
merged_df = cgss_data_df.merge(wide_df, left_on='china10_cntygb', right_on='CNTYGB_2010', how='left')
merged_df.drop(columns=['CNTYGB_2010'], inplace=True)
merged_df

,id_old,id_str,surveyyear,id_surveyyear,s41,s42,s43,cntyname,china10_cntygb,s44,...,gpt_proportion_帝王_C,gpt_proportion_帝王_D,gpt_proportion_遵守_A,gpt_proportion_遵守_B,gpt_proportion_遵守_C,gpt_proportion_遵守_D,gpt_proportion_结拜_A,gpt_proportion_结拜_B,gpt_proportion_结拜_C,gpt_proportion_结拜_D
0,6132.0,6132,2015,6132_2015,1.0,1.0,1.0,上海市徐汇区,310104,1.0,...,0.0,0.791667,0.166667,0.041667,0.25,0.541667,0.083333,0.0,0.0,0.916667
1,10915.0,10915,2012,10915_2012,1.0,1.0,1.0,上海市徐汇区,310104,1.0,...,0.0,0.791667,0.166667,0.041667,0.25,0.541667,0.083333,0.0,0.0,0.916667
2,10328.0,10328,2013,10328_2013,1.0,1.0,1.0,上海市徐汇区,310104,1.0,...,0.0,0.791667,0.166667,0.041667,0.25,0.541667,0.083333,0.0,0.0,0.916667
3,11566.0,11566,2010,11566_2010,1.0,1.0,1.0,上海市徐汇区,310104,1.0,...,0.0,0.791667,0.166667,0.041667,0.25,0.541667,0.083333,0.0,0.0,0.916667
4,346.0,346,2013,346_2013,1.0,1.0,1.0,上海市徐汇区,310104,1.0,...,0.0,0.791667,0.166667,0.041667,0.25,0.541667,0.083333,0.0,0.0,0.916667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51569,2002.0,2002,2013,2002_2013,31.0,89.0,134.0,黑龙江省齐齐哈尔市克山县,230229,368.0,...,0.0,1.000000,0.000000,0.000000,0.00,1.000000,0.500000,0.0,0.0,0.500000
51570,5096.0,5096,2010,5096_2010,31.0,89.0,134.0,黑龙江省齐齐哈尔市克山县,230229,368.0,...,0.0,1.000000,0.000000,0.000000,0.00,1.000000,0.500000,0.0,0.0,0.500000
51571,2022.0,2022,2013,2022_2013,31.0,89.0,134.0,黑龙江省齐齐哈尔市克山县,230229,370.0,...,0.0,1.000000,0.000000,0.000000,0.00,1.000000,0.500000,0.0,0.0,0.500000
51572,5100.0,5100,2010,5100_2010,31.0,89.0,134.0,黑龙江省齐齐哈尔市克山县,230229,368.0,...,0.0,1.000000,0.000000,0.000000,0.00,1.000000,0.500000,0.0,0.0,0.500000


In [63]:
# 定义保存路径
output_csv_path = '/Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/result/250203cgss_gpt_county_level_mean.csv'

# 保存为 CSV 文件
merged_df.to_csv(output_csv_path, index=False, encoding='utf-8-sig')

print(f"合并后的数据已保存到: {output_csv_path}")

合并后的数据已保存到: /Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/result/250203cgss_gpt_county_level_mean.csv


制作bert相关的变量

1. 先把这两个表给合并了

In [64]:
import pandas as pd

# Define file paths
file_path1 = "/Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/result/250203similarity_scores.csv"
file_path2 = "/Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/result/250108similarity_scores.csv"

# Read CSV files
df1 = pd.read_csv(file_path1)
df2 = pd.read_csv(file_path2)

In [65]:
df1.head()

,Unnamed: 0.1,Unnamed: 0,rowid,name,cailu,provgb,citygb,cntygb,area,geocode,...,结拜（Sworn Brotherhood）_Specific_相似度,诚实（Honesty）_Specific_中立_相似度,帝王（Emperors）_Generic_相似度,诚实（Honesty）_Specific_反面_相似度,佛教（Buddhism）_Specific_相似度,儒家（Confucianism）_Specific_相似度,帝王（Emperors）_Specific_相似度,规则遵循（Rule-Following）_Generic_相似度,家庭纽带（Family Ties）_Generic_相似度,规则遵循（Rule-Following）_Specific_反面_相似度
0,0,0,1,04946__322老汉的宝物,missing values,missing valuesd,missing valuesd,missing valuesd,青海循化县,"{'lng': 116.413384, 'lat': 39.910925}",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1,2,04743__121造过麦山的白土城,missing values,63,6321,632122,青海民和县,"{'lng': 102.83639, 'lat': 36.325561}",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2,3,05119__496伊斯玛悔过,林昌林,63,6301,630100,青海西宁市,"{'lng': 101.78445, 'lat': 36.623385}",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,3,4,05049__425三姊妹,李友楼,missing valuesd,missing valuesd,missing valuesd,青海互助县,"{'lng': 101.964569, 'lat': 36.850022}",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,4,5,04849__225老鼠的智慧,missing values,63,6323,632321,青海同仁县,"{'lng': 116.413384, 'lat': 39.910925}",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [68]:
import pandas as pd

# Define file paths
file_path1 = "/Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/result/250203similarity_scores.csv"
file_path2 = "/Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/result/250108similarity_scores.csv"

# Read CSV files
df1 = pd.read_csv(file_path1)
df2 = pd.read_csv(file_path2)

# Ensure 'name' column exists in both dataframes
if 'name' not in df1.columns or 'name' not in df2.columns:
    raise ValueError("The 'name' column is required in both dataframes.")

# Select the columns from df2 starting from the 34th column
df2_subset = df2.iloc[:, 33:]  # Since Python uses 0-based indexing, 34th column is index 33

# Merge based on 'name' column
df_merged = df1.merge(df2[['name']], on='name', how='left')  # Ensure all names exist
for col in df2_subset.columns:
    if col in df1.columns:  # Only update existing columns
        df_merged[col] = df1[col].combine_first(df2_subset[col])

In [69]:
df_merged

,Unnamed: 0.1,Unnamed: 0,rowid,name,cailu,provgb,citygb,cntygb,area,geocode,...,结拜（Sworn Brotherhood）_Specific_相似度,诚实（Honesty）_Specific_中立_相似度,帝王（Emperors）_Generic_相似度,诚实（Honesty）_Specific_反面_相似度,佛教（Buddhism）_Specific_相似度,儒家（Confucianism）_Specific_相似度,帝王（Emperors）_Specific_相似度,规则遵循（Rule-Following）_Generic_相似度,家庭纽带（Family Ties）_Generic_相似度,规则遵循（Rule-Following）_Specific_反面_相似度
0,0,0,1,04946__322老汉的宝物,missing values,missing valuesd,missing valuesd,missing valuesd,青海循化县,"{'lng': 116.413384, 'lat': 39.910925}",...,0.768833,0.790521,0.605243,0.683087,0.711533,0.675842,0.702817,0.708157,0.705420,0.710773
1,1,1,2,04743__121造过麦山的白土城,missing values,63,6321,632122,青海民和县,"{'lng': 102.83639, 'lat': 36.325561}",...,0.713582,0.730958,0.682569,0.597679,0.724805,0.724853,0.764424,0.717006,0.701990,0.772876
2,2,2,3,05119__496伊斯玛悔过,林昌林,63,6301,630100,青海西宁市,"{'lng': 101.78445, 'lat': 36.623385}",...,0.796973,0.773043,0.563120,0.752566,0.789030,0.704880,0.624486,0.673596,0.743632,0.743133
3,3,3,4,05049__425三姊妹,李友楼,missing valuesd,missing valuesd,missing valuesd,青海互助县,"{'lng': 101.964569, 'lat': 36.850022}",...,0.781996,0.748184,0.609399,0.668093,0.689730,0.692779,0.673418,0.659919,0.677790,0.646570
4,4,4,5,04849__225老鼠的智慧,missing values,63,6323,632321,青海同仁县,"{'lng': 116.413384, 'lat': 39.910925}",...,0.736825,0.763945,0.620833,0.636208,0.703185,0.707708,0.698389,0.676812,0.670092,0.705434
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19791,19791,19791,19792,19791__637麻糊涂,樊树峥,NaN,NaN,NaN,内蒙古汉族,NaN,...,0.706295,0.761495,0.650147,0.771042,0.739502,0.756244,0.617996,0.672694,0.753077,0.666594
19792,19792,19792,19793,19792__638你说念啥就念啥,张少阳,NaN,NaN,NaN,内蒙古汉族,NaN,...,0.707913,0.749443,0.535181,0.637158,0.642557,0.679803,0.603839,0.642032,0.609753,0.647312
19793,19793,19793,19794,19793__639白字县令,张少阳,NaN,NaN,NaN,内蒙古汉族,NaN,...,0.553520,0.578634,0.477106,0.572226,0.509254,0.573184,0.548120,0.538769,0.566650,0.550632
19794,19794,19794,19795,19794__640吃 鸡,王俊才,NaN,NaN,NaN,内蒙古汉族,NaN,...,0.630131,0.679420,0.545710,0.666985,0.667030,0.674544,0.537351,0.620699,0.693955,0.653600


In [70]:
output_path = "/Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/result/250204similarity_scores.csv"
df_merged.to_csv(output_path, index=False)


In [67]:
df2.columns

Index(['rowid', 'name', 'cailu', 'provgb', 'citygb', 'cntygb', 'area',
       'geocode', 'PROVGB_1953', 'CITYGB_1953', 'CNTYGB_1953', 'PROVGB_1964',
       'CITYGB_1964', 'CNTYGB_1964', 'PROVGB_1982', 'CITYGB_1982',
       'CNTYGB_1982', 'PROVGB_1990a', 'CITYGB_1990a', 'CNTYGB_1990a',
       'PROVGB_2000', 'CITYGB_2000', 'CNTYGB_2000', 'PROVGB_2010',
       'CITYGB_2010', 'CNTYGB_2010', 'new_name', 'area_group', 'centroid_1982',
       'centroid_1990', 'centroid_2000', 'combined_centroid', 'matched_gbcode',
       '规则遵循（Rule-Following）_Specific_正面_相似度', '信任（Trust）_Generic_相似度',
       '宗族（Clan）_Generic_相似度', '平等（Equality）_Specific_中立_相似度',
       '诚实（Honesty）_Specific_反面_相似度', '家庭纽带（Family Ties）_Specific_相似度',
       '儒家（Confucianism）_Generic_相似度', '慈善（Charity）_Specific_正面_相似度',
       '馈赠（Gift-Giving）_Generic_相似度', '长寿（Longevity）_Generic_相似度',
       '帝王（Emperors）_Generic_相似度', '诚实（Honesty）_Specific_中立_相似度',
       '平等（Equality）_Specific_反面_相似度',
       '儒家中的仁义（Benevolence and Justice

In [72]:
import pandas as pd

# 文件路径
similarity_scores_path = '/Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/result/250204similarity_scores.csv'
spatial_join_path = "/Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/result/250204地图匹配_spatial_join_2010_no_inconsistency.xlsx"
cgss_data_path = "/Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/result/241220to_use/CGSS2010-2015_withoutvarlabel.dta"

# 读取文件到DataFrame
bert_similarity_scores_df = pd.read_csv(similarity_scores_path)
spatial_join_df = pd.read_excel(spatial_join_path)
cgss_data_df= pd.read_stata(cgss_data_path)

In [73]:
import pandas as pd
# 筛选列名包含“相似度”的列
similarity_columns = [col for col in bert_similarity_scores_df.columns if '相似度' in col]
# 按 CNTYGB_2010_y 分组并计算包含“相似度”列的平均值
grouped_mean = bert_similarity_scores_df.groupby('CNTYGB_2010')[similarity_columns].mean()
grouped_mean

,慈善（Charity）_Specific_反面_相似度,诚实（Honesty）_Generic_相似度,儒家中的仁义（Benevolence and Justice in Confucianism）_Generic_相似度,长寿（Longevity）_Generic_相似度,诚实（Honesty）_Specific_正面_相似度,儒家中的忠诚（Loyalty in Confucianism）_Generic_相似度,结拜（Sworn Brotherhood）_Generic_相似度,慈善（Charity）_Generic_相似度,规则遵循（Rule-Following）_Specific_正面_相似度,慈善（Charity）_Specific_中立_相似度,...,结拜（Sworn Brotherhood）_Specific_相似度,诚实（Honesty）_Specific_中立_相似度,帝王（Emperors）_Generic_相似度,诚实（Honesty）_Specific_反面_相似度,佛教（Buddhism）_Specific_相似度,儒家（Confucianism）_Specific_相似度,帝王（Emperors）_Specific_相似度,规则遵循（Rule-Following）_Generic_相似度,家庭纽带（Family Ties）_Generic_相似度,规则遵循（Rule-Following）_Specific_反面_相似度
CNTYGB_2010,,,,,,,,,,,,,,,,,,,,,
110101.0,0.648439,0.677208,0.661452,0.691354,0.686725,0.632888,0.619564,0.681010,0.693669,0.605148,...,0.707329,0.719036,0.621418,0.687689,0.666983,0.649267,0.682155,0.686385,0.681940,0.680770
110102.0,0.667948,0.689251,0.663873,0.704670,0.688213,0.622838,0.625920,0.691763,0.694088,0.624265,...,0.699159,0.721163,0.607228,0.683743,0.686837,0.658359,0.667515,0.696898,0.704607,0.685437
110103.0,0.666936,0.693649,0.665961,0.688712,0.682228,0.621160,0.624310,0.693388,0.675598,0.629941,...,0.702594,0.716650,0.600927,0.678446,0.676592,0.672560,0.643838,0.669250,0.694835,0.663071
110104.0,0.673213,0.707529,0.672419,0.702806,0.700509,0.651066,0.634750,0.695235,0.686723,0.633810,...,0.710927,0.725634,0.629384,0.689792,0.674201,0.677232,0.675243,0.690660,0.700688,0.676420
110105.0,0.689805,0.724622,0.694763,0.739319,0.696360,0.668839,0.672916,0.718864,0.721604,0.651558,...,0.710950,0.740488,0.627889,0.698519,0.704376,0.683820,0.661397,0.700724,0.721596,0.699162
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
654322.0,0.676831,0.672993,0.699533,0.704036,0.631483,0.637296,0.688054,0.693668,0.689676,0.692008,...,0.719901,0.724576,0.607480,0.649545,0.714185,0.665238,0.660240,0.687082,0.678662,0.691645
654323.0,0.652240,0.654060,0.666334,0.706992,0.652651,0.653355,0.678049,0.649749,0.661726,0.634973,...,0.697388,0.715897,0.674151,0.678581,0.664385,0.662135,0.681297,0.692365,0.674429,0.645935
654324.0,0.691103,0.694056,0.704677,0.712042,0.637335,0.663908,0.669059,0.712421,0.693598,0.687511,...,0.743223,0.752959,0.631874,0.653347,0.706534,0.692129,0.653307,0.686292,0.700863,0.696997


In [74]:
# 替换列名为统一格式，去掉结尾的“_相似度”
grouped_mean.rename(
    columns=lambda x: f"bert_similarity_{x.rsplit('_相似度', 1)[0]}" if x.endswith('_相似度') else x,
    inplace=True
)

grouped_mean

,bert_similarity_慈善（Charity）_Specific_反面,bert_similarity_诚实（Honesty）_Generic,bert_similarity_儒家中的仁义（Benevolence and Justice in Confucianism）_Generic,bert_similarity_长寿（Longevity）_Generic,bert_similarity_诚实（Honesty）_Specific_正面,bert_similarity_儒家中的忠诚（Loyalty in Confucianism）_Generic,bert_similarity_结拜（Sworn Brotherhood）_Generic,bert_similarity_慈善（Charity）_Generic,bert_similarity_规则遵循（Rule-Following）_Specific_正面,bert_similarity_慈善（Charity）_Specific_中立,...,bert_similarity_结拜（Sworn Brotherhood）_Specific,bert_similarity_诚实（Honesty）_Specific_中立,bert_similarity_帝王（Emperors）_Generic,bert_similarity_诚实（Honesty）_Specific_反面,bert_similarity_佛教（Buddhism）_Specific,bert_similarity_儒家（Confucianism）_Specific,bert_similarity_帝王（Emperors）_Specific,bert_similarity_规则遵循（Rule-Following）_Generic,bert_similarity_家庭纽带（Family Ties）_Generic,bert_similarity_规则遵循（Rule-Following）_Specific_反面
CNTYGB_2010,,,,,,,,,,,,,,,,,,,,,
110101.0,0.648439,0.677208,0.661452,0.691354,0.686725,0.632888,0.619564,0.681010,0.693669,0.605148,...,0.707329,0.719036,0.621418,0.687689,0.666983,0.649267,0.682155,0.686385,0.681940,0.680770
110102.0,0.667948,0.689251,0.663873,0.704670,0.688213,0.622838,0.625920,0.691763,0.694088,0.624265,...,0.699159,0.721163,0.607228,0.683743,0.686837,0.658359,0.667515,0.696898,0.704607,0.685437
110103.0,0.666936,0.693649,0.665961,0.688712,0.682228,0.621160,0.624310,0.693388,0.675598,0.629941,...,0.702594,0.716650,0.600927,0.678446,0.676592,0.672560,0.643838,0.669250,0.694835,0.663071
110104.0,0.673213,0.707529,0.672419,0.702806,0.700509,0.651066,0.634750,0.695235,0.686723,0.633810,...,0.710927,0.725634,0.629384,0.689792,0.674201,0.677232,0.675243,0.690660,0.700688,0.676420
110105.0,0.689805,0.724622,0.694763,0.739319,0.696360,0.668839,0.672916,0.718864,0.721604,0.651558,...,0.710950,0.740488,0.627889,0.698519,0.704376,0.683820,0.661397,0.700724,0.721596,0.699162
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
654322.0,0.676831,0.672993,0.699533,0.704036,0.631483,0.637296,0.688054,0.693668,0.689676,0.692008,...,0.719901,0.724576,0.607480,0.649545,0.714185,0.665238,0.660240,0.687082,0.678662,0.691645
654323.0,0.652240,0.654060,0.666334,0.706992,0.652651,0.653355,0.678049,0.649749,0.661726,0.634973,...,0.697388,0.715897,0.674151,0.678581,0.664385,0.662135,0.681297,0.692365,0.674429,0.645935
654324.0,0.691103,0.694056,0.704677,0.712042,0.637335,0.663908,0.669059,0.712421,0.693598,0.687511,...,0.743223,0.752959,0.631874,0.653347,0.706534,0.692129,0.653307,0.686292,0.700863,0.696997


In [75]:
grouped_mean.reset_index(inplace=True)
if 'CNTYGB_2010' in grouped_mean.columns:
    # 将CNTYGB_2010_y列转换为整数类型，处理NaN值为默认值（如0）
    grouped_mean['CNTYGB_2010'] = grouped_mean['CNTYGB_2010'].fillna(0).astype(int)
else:
    print("列名 'CNTYGB_2010' 不存在，请检查数据框")
grouped_mean

,CNTYGB_2010,bert_similarity_慈善（Charity）_Specific_反面,bert_similarity_诚实（Honesty）_Generic,bert_similarity_儒家中的仁义（Benevolence and Justice in Confucianism）_Generic,bert_similarity_长寿（Longevity）_Generic,bert_similarity_诚实（Honesty）_Specific_正面,bert_similarity_儒家中的忠诚（Loyalty in Confucianism）_Generic,bert_similarity_结拜（Sworn Brotherhood）_Generic,bert_similarity_慈善（Charity）_Generic,bert_similarity_规则遵循（Rule-Following）_Specific_正面,...,bert_similarity_结拜（Sworn Brotherhood）_Specific,bert_similarity_诚实（Honesty）_Specific_中立,bert_similarity_帝王（Emperors）_Generic,bert_similarity_诚实（Honesty）_Specific_反面,bert_similarity_佛教（Buddhism）_Specific,bert_similarity_儒家（Confucianism）_Specific,bert_similarity_帝王（Emperors）_Specific,bert_similarity_规则遵循（Rule-Following）_Generic,bert_similarity_家庭纽带（Family Ties）_Generic,bert_similarity_规则遵循（Rule-Following）_Specific_反面
0,110101,0.648439,0.677208,0.661452,0.691354,0.686725,0.632888,0.619564,0.681010,0.693669,...,0.707329,0.719036,0.621418,0.687689,0.666983,0.649267,0.682155,0.686385,0.681940,0.680770
1,110102,0.667948,0.689251,0.663873,0.704670,0.688213,0.622838,0.625920,0.691763,0.694088,...,0.699159,0.721163,0.607228,0.683743,0.686837,0.658359,0.667515,0.696898,0.704607,0.685437
2,110103,0.666936,0.693649,0.665961,0.688712,0.682228,0.621160,0.624310,0.693388,0.675598,...,0.702594,0.716650,0.600927,0.678446,0.676592,0.672560,0.643838,0.669250,0.694835,0.663071
3,110104,0.673213,0.707529,0.672419,0.702806,0.700509,0.651066,0.634750,0.695235,0.686723,...,0.710927,0.725634,0.629384,0.689792,0.674201,0.677232,0.675243,0.690660,0.700688,0.676420
4,110105,0.689805,0.724622,0.694763,0.739319,0.696360,0.668839,0.672916,0.718864,0.721604,...,0.710950,0.740488,0.627889,0.698519,0.704376,0.683820,0.661397,0.700724,0.721596,0.699162
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1893,654322,0.676831,0.672993,0.699533,0.704036,0.631483,0.637296,0.688054,0.693668,0.689676,...,0.719901,0.724576,0.607480,0.649545,0.714185,0.665238,0.660240,0.687082,0.678662,0.691645
1894,654323,0.652240,0.654060,0.666334,0.706992,0.652651,0.653355,0.678049,0.649749,0.661726,...,0.697388,0.715897,0.674151,0.678581,0.664385,0.662135,0.681297,0.692365,0.674429,0.645935
1895,654324,0.691103,0.694056,0.704677,0.712042,0.637335,0.663908,0.669059,0.712421,0.693598,...,0.743223,0.752959,0.631874,0.653347,0.706534,0.692129,0.653307,0.686292,0.700863,0.696997
1896,654326,0.698561,0.674241,0.700031,0.737977,0.632593,0.630295,0.640722,0.743025,0.690318,...,0.711353,0.749798,0.561229,0.648174,0.716972,0.690157,0.639959,0.693209,0.702854,0.715045


In [76]:
# 确保列名存在
if 'china10_cntygb' in cgss_data_df.columns:
    # 将 china10_cntygb 列转换为整数类型，空值填充为 0
    cgss_data_df['china10_cntygb'] = cgss_data_df['china10_cntygb'].fillna(0).astype(int)
else:
    print("列名 'china10_cntygb' 不存在，请检查数据框")
cgss_data_df.head()

,id_old,id_str,surveyyear,id_surveyyear,s41,s42,s43,cntyname,china10_cntygb,s44,...,e112a,e112b,e113,e114,e115,z8,a59disco88,a60disco88,china10_wcd,china10_prefgb_special
0,6132.0,6132,2015,6132_2015,1.0,1.0,1.0,上海市徐汇区,310104,1.0,...,NaN,NaN,NaN,NaN,NaN,2015-10-27 16:15:00,2300.0,NaN,3101200.0,68.0
1,10915.0,10915,2012,10915_2012,1.0,1.0,1.0,上海市徐汇区,310104,1.0,...,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,3101200.0,68.0
2,10328.0,10328,2013,10328_2013,1.0,1.0,1.0,上海市徐汇区,310104,1.0,...,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,3101200.0,68.0
3,11566.0,11566,2010,11566_2010,1.0,1.0,1.0,上海市徐汇区,310104,1.0,...,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,3101200.0,68.0
4,346.0,346,2013,346_2013,1.0,1.0,1.0,上海市徐汇区,310104,1.0,...,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,3101200.0,68.0


In [77]:
# 确保需要合并的列存在
if 'CNTYGB_2010' in grouped_mean.columns and 'china10_cntygb' in cgss_data_df.columns:
    # 按照 CNTYGB_2010_y 和 china10_cntygb 列的值进行合并
    merged_df = cgss_data_df.merge(grouped_mean, left_on='china10_cntygb', right_on='CNTYGB_2010', how='left')
    # 删除 CNTYGB_2010_y 列
    merged_df.drop(columns=['CNTYGB_2010'], inplace=True)
merged_df.head()

,id_old,id_str,surveyyear,id_surveyyear,s41,s42,s43,cntyname,china10_cntygb,s44,...,bert_similarity_结拜（Sworn Brotherhood）_Specific,bert_similarity_诚实（Honesty）_Specific_中立,bert_similarity_帝王（Emperors）_Generic,bert_similarity_诚实（Honesty）_Specific_反面,bert_similarity_佛教（Buddhism）_Specific,bert_similarity_儒家（Confucianism）_Specific,bert_similarity_帝王（Emperors）_Specific,bert_similarity_规则遵循（Rule-Following）_Generic,bert_similarity_家庭纽带（Family Ties）_Generic,bert_similarity_规则遵循（Rule-Following）_Specific_反面
0,6132.0,6132,2015,6132_2015,1.0,1.0,1.0,上海市徐汇区,310104,1.0,...,0.748969,0.752417,0.616597,0.696494,0.702501,0.694874,0.65091,0.689219,0.707089,0.677486
1,10915.0,10915,2012,10915_2012,1.0,1.0,1.0,上海市徐汇区,310104,1.0,...,0.748969,0.752417,0.616597,0.696494,0.702501,0.694874,0.65091,0.689219,0.707089,0.677486
2,10328.0,10328,2013,10328_2013,1.0,1.0,1.0,上海市徐汇区,310104,1.0,...,0.748969,0.752417,0.616597,0.696494,0.702501,0.694874,0.65091,0.689219,0.707089,0.677486
3,11566.0,11566,2010,11566_2010,1.0,1.0,1.0,上海市徐汇区,310104,1.0,...,0.748969,0.752417,0.616597,0.696494,0.702501,0.694874,0.65091,0.689219,0.707089,0.677486
4,346.0,346,2013,346_2013,1.0,1.0,1.0,上海市徐汇区,310104,1.0,...,0.748969,0.752417,0.616597,0.696494,0.702501,0.694874,0.65091,0.689219,0.707089,0.677486


In [78]:
# 定义保存路径
output_csv_path = '/Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/result/250204cgss_bert_county_level_mean.csv'
# 保存为 CSV 文件
merged_df.to_csv(output_csv_path, index=False, encoding='utf-8-sig')
print(f"合并后的数据已保存到: {output_csv_path}")

合并后的数据已保存到: /Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/result/250204cgss_bert_county_level_mean.csv
